## Notebook to classify snow-covered area (SCA) and determine minimum snow elevation in PlanetScope 4-band imagery

Rainey Aberle

2022

__To-Do:__
- Extract minimum snow elevation from each SCA map using a DEM

### Import packages

In [ ]:
import os
import glob
import numpy as np
import geopandas as gpd
import rasterio as rio
from rasterio.mask import mask
from rasterio.plot import show
import earthpy.spatial as es
import matplotlib
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib.pyplot as plt
from shapely.geometry import shape
import pickle
import sys
import time

### Define settings and paths in directory

In [ ]:
# -----Define site ID (used to load classifier and in output file names)
site_ID = 'WG'

# -----Determine whether to save output figures
save_figures = True # = True to save output figures
crop_to_AOI = True # = True to crop images to AOI before calculating SCA

# -----Define desired EPSG
epsg = 32606

# -----Define paths in directory
# base directory (path to "planet-snow/")
base_path = '/Users/raineyaberle/Research/PhD/planet-snow/'
# image directory
im_path = base_path+'../study-sites/Wolverine/imagery/Planet/2021-04-20_2021-08-25/adjusted-filtered/'
# figures output directory
figures_out_path = base_path+'../study-sites/Wolverine/figures/SCA/'
# AOI shapefile full path 
AOI_fn = base_path+'../GIS_data/wolverine_RGI.shp'
# DEM tif file full path

### Load files from directory

- AOI
- PlanetScope images
- Image classifier

In [ ]:
# -----Load Area of Interest (AOI) for image cropping
AOI = gpd.read_file(AOI_fn)
# Reproject to imagery CRS if necessary
AOI = AOI.to_crs(epsg)

# -----Load image names from file
os.chdir(im_path) # change directory
im_names = glob.glob('*.tif') # load all .tif file names
im_names.sort() # sort file names by date

# -----Load image classifier and feature columns
clf_fn = base_path+'inputs-outputs/'+site_ID+'_best_classifier.sav'
clf = pickle.load(open(clf_fn, 'rb'))
feature_cols_fn = base_path+'inputs-outputs/'+site_ID+'_best_classifier_feature_cols.pkl'
feature_cols = pickle.load(open(feature_cols_fn,'rb'))

# -----Add path to functions
sys.path.insert(1, base_path+'functions/')
from classification_utils import crop_images_to_AOI, classify_image, calculate_SCA

### Classify snow, calculate SCA, determine minimum snow elevation in each image

In [ ]:
# -----Start timer
t1 = time.time()

# Initialize image dates and SCA
im_dts = [] # image datetimes
SCA = [] # [m^2]
# crop images if previously selected
if crop_to_AOI==True:
    
    # Crop images if previously selected
    cropped_im_path = crop_images_to_AOI(im_path, im_names, AOI)
    # grab cropped image names
    os.chdir(cropped_im_path) # change directory
    im_names_crop = glob.glob('*_crop.tif')
    im_names_crop.sort() # sort file names by date

    # loop through cropped images
    for im_name in im_names_crop:

        # extract datetime from image name
        im_dt = np.datetime64(im_name[0:4] + '-' + im_name[4:6] + '-' + im_name[6:8]
                              + ' ' + im_name[9:11] + ':' +im_name[11:13] + ':' + im_name[13:15])
        im_dts = im_dts + [im_dt]

        # open image
        im = rio.open(im_name)

        # classify snow
        plot_output = True
        im_x, im_y, snow, fig = classify_image(im, clf, feature_cols, plot_output)
        fig.suptitle(im_dt)
        plt.show()
        
        # calculate SCA [m^2]
        SCA = SCA + [calculate_SCA(im, snow)]

        # save figure
        if save_figures==True:
            fig.savefig(figures_out_path+im_name[0:15]+'_SCA.png', dpi=200, facecolor='white', edgecolor='none')
            print('figure saved to file')
    
    # plot SCA
    fig = plt.figure(figsize=(8,8))
    plt.scatter(im_dts, [x/1e6 for x in SCA], s=10, color='blue')
    plt.ylabel('Snow-covered area [km^2]')

# -----Stop timer
print('Time elapsed: '+str(np.round((t2-time.time())/60, 2))+' minutes')